In [1]:
from docplex.mp.model import Model
import pandas as pd
from pandas import DataFrame
import numpy as np
from cplex.callbacks import LazyConstraintCallback
import cplex
import time
from termcolor import colored
from cplex.exceptions import CplexSolverError
import os

from src.data_class import ProblemData
from src.maxmin_diversity.bisection import BisectionMethod

## The $\epsilon$ - constraint method ($\epsilon$C) to solve BpMMMDP

In [2]:
# The p-median problem with beta 

def pM_beta_mm(n,p,q,max_time,beta,mdis,epsilon):
    bqp = Model(name="BQP model")
    
    #decision variables
    yik = [(i,k) for i in range(n) for k in range(n)]
    xi = [i for i in range(n)]

    y = bqp.binary_var_dict(keys=yik,name="y")
    x = bqp.binary_var_list(keys=xi,name="x")

    #constraints
    # Single assignment
    for k in range(n):
        bqp.add_constraint(bqp.sum([y[i,k] for i in range(n)]) == 1)
        
    # p-center
    [bqp.add_constraint(bqp.sum([x[i] for i in range(n)]) == p)] 
    
    # only assign to center 
    for i in range(n):
        for k in range(n):
            bqp.add_constraint(y[i,k] <= x[i])
            
    # dispersion constraints
    [bqp.add_constraint(bqp.sum([q[i,j]*x[i]*x[j] for i in range(n) for j in range(n)]) >= beta + epsilon)]
    
    #xi+xj<=1, when dij<m
    for i in range(n):
        for j in range(i+1,n):
            if 0 < q[i,j] < mdis:
                bqp.add_constraint(x[i]+x[j] <= 1)
 
    #objective                   
    bqp.minimize(bqp.sum([q[i,k]*y[i,k] for i in range(n) for k in range(n)]))
    bqp.parameters.timelimit = max_time
    
    #solve the problem
    sol = bqp.solve() 
    
    if sol is not None:
        # Solution exists, hence problem is feasible 
        bqplb = bqp.solve_details.best_bound
        bqpub = sol.get_objective_value()
        bqpgap = bqp.solve_details.mip_relative_gap
        bqpruntime = bqp.solve_details.time

        xr = np.array(sol.get_value_list(x))
        newbeta = sum(q[i,j]*xr[i]*xr[j] for i in range(n) for j in range(n))
        return xr, bqpub, bqpruntime, newbeta
    
    else:
        # Solution does not exist, problem is infeasible
        return "infeasible"


In [3]:
# Find Pareto efficient solutions for the bi-objective p median and max-sum diversity problem 
def PM_Pareto(n,p,q,max_time,mdis,epsilon):
    start = time.time()
    num_iter = 0
    beta = -epsilon
    obj = 0
    num_pareto = 0
    num_pareto_pm_mm = 1
    runtime_pm_mm = 0
    obj_min_pm_mm = []
    obj_max_pm_mm = []
    time_list_pm_mm = []
    time_pareto_pm_mm =[]
    
    # iterations
    while runtime_pm_mm < max_time:  
        solution = pM_beta_mm(n,p,q,max_time,beta,mdis,epsilon)
        runtime_pm_mm = time.time()-start
        if solution == "infeasible":
            print(colored('Infeasibility of epsilon constraint method is achieved.','red'))
            return obj_min_pm_mm, obj_max_pm_mm, num_pareto_pm_mm, runtime_pm_mm
        else:     
            x_pm, newobj, pmruntime, newbeta = solution
            time_list_pm_mm.append(round(pmruntime,2))
            
        # look for the next Pareto optimal solution
        if newobj >= obj + epsilon: 
            obj_min_pm_mm.append(round(newobj,2))
            obj_max_pm_mm.append(round(newbeta,2))
            time_pareto_pm_mm.append(round(pmruntime,2))
            num_pareto += 1 
            num_pareto_pm_mm= num_pareto
            beta=newbeta
            obj=newobj
        if newobj < obj + epsilon:
            obj_min_pm_mm[-1]=round(newobj,2)
            obj_max_pm_mm[-1]=round(newbeta,2)
            time_pareto_pm_mm[-1]=round(pmruntime,2)
            beta=newbeta
            
        # update the iteration  
        num_iter += 1 
        
    return obj_min_pm_mm, obj_max_pm_mm, num_pareto_pm_mm, runtime_pm_mm


## The $\epsilon$ - constraint method incorporating with tangent cutting plane method ($\epsilon$C + TCP)

In [4]:
# objective function for x
def f(x, q):
    x = x.copy()
    return x.dot(q).dot(x)

# gradient f
def df(x, q):
    x = x.copy()
    return x.dot(q)

In [5]:
# Tangent cutting plane method for the bi-objective p-median and max-sum diversity problem
def PM_Cut(n,p,q,max_time,mdis,epsilon):
    start = time.time()
    num_iter = 0
    runtime = 0
    num_pareto = 0
    num_pareto_pmc_mm = 1
    LB_list_pmc_mm=[]
    beta_list_pmc_mm =[]
    time_pareto_pmc_mm =[]
    
    # Setup model
    ms = Model(name="master-p median model")#,log_output=True)
    
    #decision variables
    yik = [(i,k) for i in range(n) for k in range(n)]
    xi = [i for i in range(n)]
    y = ms.binary_var_dict(keys=yik,name="y")
    x = ms.binary_var_list(keys=xi,name="x")
    
    #constraints
    # Single assignment
    for k in range(n):
        ms.add_constraint(ms.sum([y[i,k] for i in range(n)]) == 1)
    # p-center
    [ms.add_constraint(ms.sum([x[i] for i in range(n)]) == p)] 
    # only assign to center 
    for i in range(n):
        for k in range(n):
            ms.add_constraint(y[i,k] <= x[i])
    #xi+xj<=1, when dij<=m
    for i in range(n):
        for j in range(i+1,n): 
            if 0 < q[i,j] < mdis:
                ms.add_constraint(x[i]+x[j] <= 1)
                
    #objective                   
    ms.minimize(ms.sum([q[i,k]*y[i,k] for i in range(n) for k in range(n)]))
    ms.parameters.timelimit = max_time
    
    # Solve
    ms.solve()
    solms = ms.solution
    
    # get new x
    xr = np.array(solms.get_value_list(x))
    beta = sum(q[i,j]*xr[i]*xr[j] for i in range(n) for j in range(n))
    lb0= ms.objective_value
    runtime0 = ms.solve_details.time
    runtime_pmc_mm = runtime0

    LB_list_pmc_mm.append(round(lb0,2))
    beta_list_pmc_mm.append(round(beta,2))
    time_pareto_pmc_mm.append(round(runtime0,2))

    while runtime_pmc_mm < max_time:    
        newbeta=0
        start1=time.time()
        while newbeta < beta + epsilon:         
            # update gradient
            dfx = df(xr, q)        
            # update the cutting plane
            ms.add_constraint(ms.sum(2*dfx[i] * x[i] for i in range(n)) - f(xr, q) >= beta + epsilon)        
            # solve the new model
            ms.parameters.timelimit = max_time
            ms.solve()
            solms = ms.solution
            runtime = time.time() - start
            runtime_pmc_mm = runtime
            
            if solms is None:
                print(colored('Infeasibility of cutting is achieved.','red'))
                return LB_list_pmc_mm, beta_list_pmc_mm, num_pareto_pmc_mm, runtime_pmc_mm
            else:
                # get new x
                xr = np.array(solms.get_value_list(x))
                newbeta = sum(q[i,j]*xr[i]*xr[j] for i in range(n) for j in range(n))
                lowerbound = ms.objective_value        

            # update steps
            #print(colored('Number of c_iteration:', 'green', attrs=['bold']), num_iter_c)
            #print(colored('Objective value:', 'green', attrs=['bold']), lowerbound)
            #print(colored('newbeta:', 'green', attrs=['bold']), newbeta)
            #print(colored('Solution:', 'green', attrs=['bold']), xr)

        # look for the next Pareto optimal solution
        runtime_iter = time.time() - start1              
        if lowerbound >= lb0 + epsilon: 
            LB_list_pmc_mm.append(round(lowerbound,2))
            beta_list_pmc_mm.append(round(newbeta,2))
            time_pareto_pmc_mm.append(round(runtime_iter,2))
            num_pareto += 1 
            num_pareto_pmc_mm= num_pareto+1
            beta=newbeta
            lb0=lowerbound        
        if lowerbound < lb0 + epsilon:
            LB_list_pmc_mm[-1]=round(lowerbound,2)
            beta_list_pmc_mm[-1]=round(newbeta,2)
            time_pareto_pmc_mm[-1]=round(runtime_iter,2)
            beta=newbeta    

        num_iter +=1
    
    return LB_list_pmc_mm, beta_list_pmc_mm, num_pareto_pmc_mm, runtime_pmc_mm 
            

## The $\epsilon$ - constraint method incorporating with tangent cutting plane method and Benders decompostion with Balinski method ($\epsilon$C + TCP + BD)

In [6]:
# Use Balinski method to generate dual values of subproblem in Benders decompostion

# compute lambda values
def get_lambda(x, q):
    """
    For each k, compute lambda[k] = min{ q[i,k] : x[i] = 1 }.
    Guaranteed non-empty since sum(xi) = p > 0.
    """
    lambda_l = []
    for k in range(n):
        vals = [q[i,k] for i in range(n) if x[i] > 0.5]  # consistent with upperbound_value_actual
        lambda_l.append(min(vals))
    return lambda_l

# compute pi values
def get_pi(x, q):
    """
    For inactive i (x[i] = 0), pi[i,k] = min(0, q[i,k] - lambda[k]).
    For active i (x[i] = 1), pi[i,k] = 0.
    """
    lambda_l = get_lambda(x, q)
    pi_l = np.zeros((n,n))
    for i in range(n):
        if x[i] < 0.5:
            for k in range(n):
                pi_l[i,k] = min(0, q[i,k] - lambda_l[k])

    return pi_l

# compute upper bound
def upperbound_value_mp(lambda_l):
    """
    Upper bound = sum_k lambda[k].
    """
    ub = sum(lambda_l[k] for k in range(n)) 
    
    return ub 


In [7]:
# Benders (with Balinski method) and tangent cutting plane method for the bi-objective p-median and max-sum diversity problem 
def PM_Benders_balinski(n,p,q,max_time,mdis,epsilon):
    start = time.time()
    num_iter = 0
    runtime = 0
    num_pareto = 0
    num_pareto_pmcbd_mm = 1
    minobj_list_pmcbd_mm=[]
    beta_list_pmcbd_mm =[]
    time_pareto_pmcbd_mm =[]
   
    # Setup the initial p median model
    ms = Model(name="initial-p median model")
   
    #decision variables
    yik = [(i,k) for i in range(n) for k in range(n)]
    xi = [i for i in range(n)]
    y = ms.binary_var_dict(keys=yik,name="y")
    x = ms.binary_var_list(keys=xi,name="x")
    
    #constraints
    # Single assignment
    for k in range(n):
        ms.add_constraint(ms.sum([y[i,k] for i in range(n)]) == 1)
    # p-center
    [ms.add_constraint(ms.sum([x[i] for i in range(n)]) == p)] 
    # only assign to center 
    for i in range(n):
        for k in range(n):
            ms.add_constraint(y[i,k] <= x[i])       
    #xi+xj<=1, when dij<m
    for i in range(n):
        for j in range(i+1,n):  
            if 0 < q[i,j] < mdis:
                ms.add_constraint(x[i]+x[j] <= 1)
                
    #objective                   
    ms.minimize(ms.sum([q[i,k]*y[i,k] for i in range(n) for k in range(n)]))
    ms.parameters.timelimit = max_time
    
    # Solve
    ms.solve()
    solms = ms.solution

    # get new x
    xr = np.array(solms.get_value_list(x))
    beta = sum(q[i,j]*xr[i]*xr[j] for i in range(n) for j in range(n))
    lb0= ms.objective_value
    runtime0 = ms.solve_details.time
    runtime_pmcbd_mm = runtime0

    minobj_list_pmcbd_mm.append(round(lb0,2))
    beta_list_pmcbd_mm.append(round(beta,2))
    time_pareto_pmcbd_mm.append(round(runtime0,2))
    
    # setup the initial master model
    m = Model(name = "master problem")
    
    #decision variables
    xi = [i for i in range(n)]
    thetak= [k for k in range(n)]
    x = m.binary_var_list(keys=xi,name="x")
    theta = m.continuous_var_dict(keys=thetak, name="theta", lb = 0)
    
    # add constraints
    m.add_constraint(m.sum(x[i] for i in range(n)) == p)
    #xi+xj<=1, when dij<m
    for i in range(n):
        for j in range(i+1,n):  
            if 0 < q[i,j] < mdis:
                m.add_constraint(x[i]+x[j] <= 1)
                
    # objective function
    m.minimize(m.sum(theta[k] for k in range(n)))

    while runtime < max_time:   
        UB=100000000
        LB=-10000000       
        start1=time.time()

        while UB - LB >= 0.01:
            newbeta=0
            while newbeta < beta + epsilon:      
                # update gradient
                dfx = df(xr, q) 
        
                # update the cutting plane
                m.add_constraint(m.sum(2*dfx[i] * x[i] for i in range(n)) - f(xr, q) >= beta + epsilon)
                
                # solve the new model
                m.parameters.timelimit = max_time
                m.solve()
                solm = m.solution
                
                runtime = time.time() - start
                runtime_pmcbd_mm = runtime
                                
                if solm is None:
                    print(colored('Infeasibility of Benders with balinski cuts is achieved.','red'))                    
                    return minobj_list_pmcbd_mm, beta_list_pmcbd_mm, num_pareto_pmcbd_mm, runtime_pmcbd_mm  
                else:
                    # get new x
                    xr = np.array(solm.get_value_list(x))
                    newbeta = sum(q[i,j]*xr[i]*xr[j] for i in range(n) for j in range(n))
                    lowerbound = m.objective_value

            # balinski method to get lambda and pi and upperbound
            lambda_l = get_lambda(xr, q)
            pi_l = get_pi(xr, q)
            upperbound = upperbound_value_mp(lambda_l)
            
            # Update the bounds
            LB = max(LB,lowerbound) 
            UB = min(UB,upperbound)

            # add the optimality cut to the master model
            for k in range(n):
                m.add_constraint(theta[k] >= lambda_l[k] + m.sum(pi_l[i,k]*x[i] for i in range(n)))

        # look for the next Pareto optimal solution
        runtime_iter= time.time() - start1    
        if lowerbound >= lb0 + epsilon: 
            minobj_list_pmcbd_mm.append(round(lowerbound,2))
            beta_list_pmcbd_mm.append(round(newbeta,2))
            time_pareto_pmcbd_mm.append(round(runtime_iter,2))
            num_pareto += 1 
            num_pareto_pmcbd_mm = num_pareto+1
            beta=newbeta
            lb0=lowerbound        
        if lowerbound < lb0 + epsilon:
            minobj_list_pmcbd_mm[-1]=round(lowerbound,2)
            beta_list_pmcbd_mm[-1]=round(newbeta,2)
            time_pareto_pmcbd_mm[-1]=round(runtime_iter,2)
            beta=newbeta    

        # update the iteration
        num_iter +=1
    
    return minobj_list_pmcbd_mm, beta_list_pmcbd_mm, num_pareto_pmcbd_mm, runtime_pmcbd_mm


## The $\epsilon$ - constraint method incorporating with tangent cutting plane method and branch-and-cut version of Benders decompostion ($\epsilon$C + TCP + BC)

In [8]:
# The subproblem in Benders decompostion 
def sub_problem(x,q):    
    # setup model
    sub = Model(name = "sub problem")

    # set continuous variables
    y_key = [(i,k) for i in range(n) 
                   for k in range(n)]
    y = sub.continuous_var_dict(keys=y_key, name="y",lb=0)

    # add constraints
    constraint_list1 = []
    constraint_index1 = []
    for k in range(n):
        constraint_list1.append(sub.add_constraint(sub.sum(y[i,k] for i in range(n)) == 1))
        constraint_index1.append(k)

    constraint_list2 = []
    constraint_index2 = []                           
    for i in range(n):
        for k in range(n):
            constraint_list2.append(sub.add_constraint(y[i,k] <= x[i]))
            constraint_index2.append((i,k))

    # objective function
    sub.minimize(sub.sum(q[i,k]*y[i,k] for i in range(n) for k in range(n)))

    sub.solve()

    solsub = sub.solution   
    sub_obj = sub.objective_value
    
    lambda_val = { constraint_index1[i] : constraint_list1[i].dual_value for i in range(len(constraint_index1))}
    lambda_l=[]
    for k in range(n):
        lambda_l.append(lambda_val[k])
               
    pi_val = { constraint_index2[i] : constraint_list2[i].dual_value for i in range(len(constraint_index2))}
    pi_l = np.zeros((n,n))
    for i in range(n):
        for k in range(n):
            pi_l[i,k] = pi_val[i,k]
                        
    y_ik = np.zeros((n,n)) 
    for i in range(n):
        for k in range(n):
            y_ik[i,k] = solsub.get_value("y_"+str(i)+"_"+str(k))
                
    return lambda_l, pi_l, sub_obj, y_ik
   

In [9]:
class BendersLazyCallback(LazyConstraintCallback):
    def __init__(self, env):
        super().__init__(env)

    def set_data(self, x_indices, theta_indices, q, beta, epsilon):
        self.x_indices = x_indices
        self.theta_indices = theta_indices
        self.q = q
        self.beta = beta
        self.epsilon = epsilon
        self.n = len(x_indices)

    def __call__(self):
        x_vals = np.array(self.get_values(self.x_indices))
        if not np.all(np.abs(x_vals - np.round(x_vals)) < 1e-6):
            return

        lambda_l, pi_l, upperbound, _ = sub_problem(x_vals, self.q)
        dfx = df(x_vals, self.q)
        fx_val = f(x_vals, self.q)

        # Add Benders cuts
        for k in range(self.n):
            expr = [1.0] + [-pi_l[i][k] for i in range(self.n)]
            ind = [self.theta_indices[k]] + [self.x_indices[i] for i in range(self.n)]
            self.add(constraint=cplex.SparsePair(ind=ind, val=expr), sense="G", rhs=lambda_l[k])

        # Add tangent cut
        grad_expr = [2 * dfx[i] for i in range(self.n)]
        self.add(
            constraint=cplex.SparsePair(ind=self.x_indices, val=grad_expr),
            sense="G",
            rhs=self.beta + self.epsilon + fx_val
        )


In [10]:
def branch_and_cut(n, p, q, max_time, beta, mdis, epsilon):
    start_time = time.time()
    m = cplex.Cplex()
    m.set_problem_type(cplex.Cplex.problem_type.MILP)
    m.parameters.timelimit.set(max_time)
    m.parameters.preprocessing.presolve.set(0)
    m.parameters.mip.strategy.search.set(1)

    # Variables: x[0..n-1] binary, theta[0..n-1] continuous
    x_names = [f"x_{i}" for i in range(n)]
    theta_names = [f"theta_{i}" for i in range(n)]

    m.variables.add(names=x_names, types=["B"] * n)
    m.variables.add(names=theta_names, types=["C"] * n, lb=[0.0] * n)

    x_indices = list(range(n))
    theta_indices = list(range(n, 2 * n))

    # Constraint: sum(x) == p
    m.linear_constraints.add(
        lin_expr=[cplex.SparsePair(ind=x_indices, val=[1.0] * n)],
        senses=["E"],
        rhs=[p]
    )

    # Conflict constraints: x_i + x_j <= 1 when q[i,j] < mdis
    for i in range(n):
        for j in range(i + 1, n):
            if 0 < q[i, j] < mdis:
                m.linear_constraints.add(
                    lin_expr=[cplex.SparsePair(ind=[x_indices[i], x_indices[j]], val=[1.0, 1.0])],
                    senses=["L"],
                    rhs=[1.0]
                )

    # Objective: minimize sum(theta)
    obj = [0.0] * n + [1.0] * n
    m.objective.set_sense(m.objective.sense.minimize)
    m.objective.set_linear(list(zip(range(2 * n), obj)))
    
    # Callback
    cb = m.register_callback(BendersLazyCallback)
    cb.set_data(x_indices, theta_indices, q, beta, epsilon)
    
    # Solve safely
    try:
        m.solve()
    except CplexSolverError as e:
        return "infeasible"

    runtime = time.time() - start_time

    # Now safely extract solution values
    try:
        xr = np.array(m.solution.get_values(x_names))
        bqplb = m.solution.get_objective_value()
        bqpub = m.solution.MIP.get_best_objective()
        newbeta = sum(q[i, j] * xr[i] * xr[j] for i in range(n) for j in range(n))
        bqpgap = 100 * (bqpub - bqplb) / abs(bqpub) if abs(bqpub) > 1e-6 else 0
        return xr, bqplb, bqpub, runtime, newbeta, bqpgap
    except CplexSolverError as e:
        return "infeasible"


In [11]:
# Branch and cut version of Benders decompostion for the bi-objective p median and max-sum diversity problem 

def PM_branch_and_cut(n,p,q,max_time,mdis,epsilon):
    start = time.time()
    num_iter = 0
    runtime = 0
    beta = -epsilon
    obj = 0
    num_pareto = 0
    num_pareto_pm_mm = 0
    runtime_pm_mm = 0
    obj_min_pm_mm = []
    obj_max_pm_mm = []
    time_list_pm_mm = []
    time_pareto_pm_mm =[]
    
    # iterations
    while runtime < max_time:
        solution = branch_and_cut(n, p, q, max_time, beta, mdis, epsilon)
        runtime=time.time()-start
        runtime_pm_mm = runtime 
        if solution == "infeasible":
            print(colored('Infeasibility of branch and cut is achieved.','red'))
            return obj_min_pm_mm, obj_max_pm_mm, num_pareto_pm_mm, runtime_pm_mm
        else:     
            x_pm, newobj, bqpub, pmruntime, newbeta, bqpgap = solution
            time_list_pm_mm.append(round(pmruntime,2))
            
        # look for the next Pareto optimal solution
        if newobj >= obj + epsilon: 
            obj_min_pm_mm.append(round(newobj,2))
            obj_max_pm_mm.append(round(newbeta,2))
            time_pareto_pm_mm.append(round(pmruntime,2))
            num_pareto += 1 
            num_pareto_pm_mm= num_pareto
            beta=newbeta
            obj=newobj
        if newobj < obj + epsilon:
            obj_min_pm_mm[-1]=round(newobj,2)
            obj_max_pm_mm[-1]=round(newbeta,2)
            time_pareto_pm_mm[-1]=round(pmruntime,2)
            beta=newbeta
        
        num_iter += 1 
        
    return obj_min_pm_mm, obj_max_pm_mm, num_pareto_pm_mm, runtime_pm_mm


## The $\epsilon$ - constraint method incorporating with tangent cutting plane method and the classical Benders decompostion

In [12]:
# Benders and tangent cutting plane method for the bi-objective p-median and max-sum with max-min 

def PM_Benders(n,p,q,max_time,mdis,epsilon):
    start = time.time()
    num_iter = 0
    runtime = 0
    num_pareto = 0
    num_pareto_pmcbd_mm = 1
    minobj_list_pmcbd_mm=[]
    beta_list_pmcbd_mm =[]
    time_pareto_pmcbd_mm =[]
   
    # Setup the initial p median model
    ms = Model(name="initial-p median model")
   
    #decision variables
    yik = [(i,k) for i in range(n) for k in range(n)]
    xi = [i for i in range(n)]
    y = ms.binary_var_dict(keys=yik,name="y")
    x = ms.binary_var_list(keys=xi,name="x")
    
    #constraints
    # Single assignment
    for k in range(n):
        ms.add_constraint(ms.sum([y[i,k] for i in range(n)]) == 1)
    # p-center
    [ms.add_constraint(ms.sum([x[i] for i in range(n)]) == p)] 
    # only assign to center 
    for i in range(n):
        for k in range(n):
            ms.add_constraint(y[i,k] <= x[i])       
    #xi+xj<=1, when dij<m
    for i in range(n):
        for j in range(i+1,n):  
            if 0 < q[i,j] < mdis:
                ms.add_constraint(x[i]+x[j] <= 1)
                
    #objective                   
    ms.minimize(ms.sum([q[i,k]*y[i,k] for i in range(n) for k in range(n)]))
    ms.parameters.timelimit = max_time
    
    # Solve
    ms.solve()
    solms = ms.solution

    # get new x
    xr = np.array(solms.get_value_list(x))
    beta = sum(q[i,j]*xr[i]*xr[j] for i in range(n) for j in range(n))
    lb0= ms.objective_value
    runtime0 = ms.solve_details.time
    runtime_pmcbd_mm = runtime0

    minobj_list_pmcbd_mm.append(round(lb0,2))
    beta_list_pmcbd_mm.append(round(beta,2))
    time_pareto_pmcbd_mm.append(round(runtime0,2))
    
    # setup the initial master model
    m = Model(name = "master problem")
    
    #decision variables
    xi = [i for i in range(n)]
    thetak= [k for k in range(n)]
    x = m.binary_var_list(keys=xi,name="x")
    theta = m.continuous_var_dict(keys=thetak, name="theta", lb = 0)
    
    # add constraints
    m.add_constraint(m.sum(x[i] for i in range(n)) == p)
    #xi+xj<=1, when dij<m
    for i in range(n):
        for j in range(i+1,n):  
            if 0 < q[i,j] < mdis:
                m.add_constraint(x[i]+x[j] <= 1)
                
    # objective function
    m.minimize(m.sum(theta[k] for k in range(n)))

    while runtime < max_time:   
        UB=100000000
        LB=-10000000       
        start1=time.time()

        while UB - LB >= 0.01:
            newbeta=0
            while newbeta < beta + epsilon:      
                # update gradient
                dfx = df(xr, q) 
                # update the cutting plane
                m.add_constraint(m.sum(2*dfx[i] * x[i] for i in range(n)) - f(xr, q) >= beta + epsilon)
                # solve the new model
                m.parameters.timelimit = max_time
                m.solve()
                solm = m.solution
                runtime = time.time() - start
                runtime_pmcbd_mm = runtime
                                
                if solm is None:
                    print(colored('Infeasibility of Benders is achieved.','red'))                    
                    return minobj_list_pmcbd_mm, beta_list_pmcbd_mm, num_pareto_pmcbd_mm, runtime_pmcbd_mm  
                else:
                    # get new x
                    xr = np.array(solm.get_value_list(x))
                    newbeta = sum(q[i,j]*xr[i]*xr[j] for i in range(n) for j in range(n))
                    lowerbound = m.objective_value

            # Generate dual values and upperbound
            lambda_l, pi_l, upperbound, y_ik = sub_problem(xr,q)
            
            # Update the bounds
            LB = max(LB,lowerbound) 
            UB = min(UB,upperbound)

            # add the optimality cut to the model
            for k in range(n):
                m.add_constraint(theta[k] >= lambda_l[k] + m.sum(pi_l[i,k]*x[i] for i in range(n)))

        # look for the next Pareto optimal solution
        runtime_iter= time.time() - start1    
        if lowerbound >= lb0 + epsilon: 
            minobj_list_pmcbd_mm.append(round(lowerbound,2))
            beta_list_pmcbd_mm.append(round(newbeta,2))
            time_pareto_pmcbd_mm.append(round(runtime_iter,2))
            num_pareto += 1 
            num_pareto_pmcbd_mm = num_pareto+1
            beta=newbeta
            lb0=lowerbound        
        if lowerbound < lb0 + epsilon:
            minobj_list_pmcbd_mm[-1]=round(lowerbound,2)
            beta_list_pmcbd_mm[-1]=round(newbeta,2)
            time_pareto_pmcbd_mm[-1]=round(runtime_iter,2)
            beta=newbeta    

        num_iter +=1
    
    return minobj_list_pmcbd_mm, beta_list_pmcbd_mm, num_pareto_pmcbd_mm, runtime_pmcbd_mm


## The $\epsilon$ - constraint method incorporating with tangent cutting plane method and Benders decompostion with Pareto optimal optimality cuts

In [13]:
def sub_problem_original(x,q):    
    # setup model
    sub = Model(name = "sub problem")

    # set continuous variables
    y_key = [(i,k) for i in range(n) 
                   for k in range(n)]
    y = sub.continuous_var_dict(keys=y_key, name="y",lb=0)

    # add constraints
    for k in range(n):
        sub.add_constraint(sub.sum(y[i,k] for i in range(n)) >= 1)
                          
    for i in range(n):
        for k in range(n):
            sub.add_constraint(y[i,k] <= x[i])

    # objective function
    sub.minimize(sub.sum(q[i,k]*y[i,k] for i in range(n) for k in range(n)))

    sub.solve()

    solsub = sub.solution   
    sub_obj = sub.objective_value
                        
    y_ik = np.zeros((n,n)) 
    for i in range(n):
        for k in range(n):
            y_ik[i,k] = solsub.get_value("y_"+str(i)+"_"+str(k))
                
    return sub_obj, y_ik

In [14]:
# The dual problem of the subproblem in Benders decomopstion

def dual_sub_problem(x, q):
    dual = Model(name="dual_subproblem")

    # Dual variables
    lambda_k = dual.continuous_var_dict(keys=range(n), name="lambda", lb=0)
    pi_key = [(i, k) for i in range(n) for k in range(n)]
    pi_ik = dual.continuous_var_dict(keys=pi_key, name="pi", lb=0)

    # Constraints: lambda_k[k] - pi_ik[i,k] <= q[i,k]
    for (i, k) in pi_key:
        dual.add_constraint(lambda_k[k] - pi_ik[i, k] <= q[i][k])

    # Objective: maximize sum(lambda_k) - sum(pi_ik * x_i)
    dual.maximize(
        dual.sum(lambda_k[k] for k in range(n)) -
        dual.sum(pi_ik[i, k] * x[i] for (i, k) in pi_key)
    )
    
    # Solve
    dual.solve()

    # Extract solution
    sol = dual.solution
    dual_obj = dual.objective_value

    lambda_l = np.array([sol.get_value("lambda_" + str(k)) for k in range(n)])
    pi_l = np.zeros((n, n))
    for (i, k) in pi_key:
        pi_l[i, k] = sol.get_value("pi_" + str(i) + "_" + str(k))

    return lambda_l, pi_l, dual_obj


In [15]:
# The additional dual problem of the subproblem in Benders decomopstion for finding non-dominated dual solutions

def dual_pareto_optimal(x, x_core, q, theta_star):
    dual = Model(name="dual_pareto_optimal")

    # Dual variables
    lambda_k = dual.continuous_var_dict(keys=range(n), name="lambda", lb=0)
    pi_key = [(i, k) for i in range(n) for k in range(n)]
    pi_ik = dual.continuous_var_dict(keys=pi_key, name="pi", lb=0)

    # Constraints: lambda_k[k] - pi_ik[i,k] <= q[i][k]
    for (i, k) in pi_key:
        dual.add_constraint(lambda_k[k] - pi_ik[i, k] <= q[i][k])

    # Constraint: fixed subproblem objective value
    dual.add_constraint(
        dual.sum(lambda_k[k] for k in range(n)) -
        dual.sum(pi_ik[i, k] * x[i] for (i, k) in pi_key) == theta_star
    )

    # Objective: maximize violation at core point
    dual.maximize(
        dual.sum(lambda_k[k] for k in range(n)) -
        dual.sum(pi_ik[i, k] * x_core[i] for (i, k) in pi_key)
    )

    # Solve
    dual.solve()

    # Extract solution
    sol = dual.solution
    lambda_l = np.array([sol.get_value("lambda_" + str(k)) for k in range(n)])
    pi_l = np.zeros((n, n))
    for (i, k) in pi_key:
        pi_l[i, k] = sol.get_value("pi_" + str(i) + "_" + str(k))

    return lambda_l, pi_l


In [16]:
# Benders with Pareto optimal cuts and tangent cutting plane method for the bi-objective p-median and max-sum diversity problem

def PM_Benders_optcuts(n,p,q,max_time,mdis,epsilon):
    start = time.time()
    num_iter = 0
    runtime = 0
    num_pareto = 0
    num_pareto_pmcbd_mm = 1
    minobj_list_pmcbd_mm=[]
    beta_list_pmcbd_mm =[]
    time_pareto_pmcbd_mm =[]
   
    # Setup the initial p median model
    ms = Model(name="initial-p median model")
   
    #decision variables
    yik = [(i,k) for i in range(n) for k in range(n)]
    xi = [i for i in range(n)]
    y = ms.binary_var_dict(keys=yik,name="y")
    x = ms.binary_var_list(keys=xi,name="x")
    
    #constraints
    # Single assignment
    for k in range(n):
        ms.add_constraint(ms.sum([y[i,k] for i in range(n)]) == 1)
    # p-center
    [ms.add_constraint(ms.sum([x[i] for i in range(n)]) == p)] 
    # only assign to center 
    for i in range(n):
        for k in range(n):
            ms.add_constraint(y[i,k] <= x[i])       
    #xi+xj<=1, when dij<m
    for i in range(n):
        for j in range(i+1,n):  
            if 0 < q[i,j] < mdis:
                ms.add_constraint(x[i]+x[j] <= 1)
                
    #objective                   
    ms.minimize(ms.sum([q[i,k]*y[i,k] for i in range(n) for k in range(n)]))
    ms.parameters.timelimit = max_time
    
    # Solve
    ms.solve()
    solms = ms.solution

    # get new x
    xr = np.array(solms.get_value_list(x))
    beta = sum(q[i,j]*xr[i]*xr[j] for i in range(n) for j in range(n))
    lb0= ms.objective_value
    runtime0 = ms.solve_details.time
    runtime_pmcbd_mm = runtime0

    minobj_list_pmcbd_mm.append(round(lb0,2))
    beta_list_pmcbd_mm.append(round(beta,2))
    time_pareto_pmcbd_mm.append(round(runtime0,2))
    
    # setup the initial master model
    m = Model(name = "master problem")
    
    #decision variables
    xi = [i for i in range(n)]
    thetak= [k for k in range(n)]
    x = m.binary_var_list(keys=xi,name="x")
    theta = m.continuous_var_dict(keys=thetak, name="theta", lb = 0)
    
    # add constraints
    m.add_constraint(m.sum(x[i] for i in range(n)) == p)
    #xi+xj<=1, when dij<m
    for i in range(n):
        for j in range(i+1,n):  
            if 0 < q[i,j] < mdis:
                m.add_constraint(x[i]+x[j] <= 1)
                
    # objective function
    m.minimize(m.sum(theta[k] for k in range(n)))

    while runtime < max_time:   
        UB=100000000
        LB=-10000000
        alpha_weight = 0.5
        # Initial core point
        x_core = np.full(n, 0.5)
        start1=time.time()
        
        while UB - LB >= 0.01:
            newbeta=0
            while newbeta < beta + epsilon:      
                # update gradient
                dfx = df(xr, q) 
                # update the cutting plane
                m.add_constraint(m.sum(2*dfx[i] * x[i] for i in range(n)) - f(xr, q) >= beta + epsilon)
                # solve the new model
                m.parameters.timelimit = max_time
                m.solve()
                solm = m.solution
                runtime = time.time() - start
                runtime_pmcbd_mm = runtime
                                
                if solm is None:
                    print(colored('Infeasibility of Benders with optcuts is achieved.','red'))                    
                    return minobj_list_pmcbd_mm, beta_list_pmcbd_mm, num_pareto_pmcbd_mm, runtime_pmcbd_mm  
                else:
                    # get new x
                    xr = np.array(solm.get_value_list(x))
                    newbeta = sum(q[i,j]*xr[i]*xr[j] for i in range(n) for j in range(n))
                    lowerbound = m.objective_value
            
            # Generate the Pareto-optimal dual solutions
            upperbound, y_ik = sub_problem_original(xr,q)
            x_core = alpha_weight * x_core + (1 - alpha_weight) * xr 
            lambda_l, pi_l = dual_pareto_optimal(xr, x_core, q, upperbound)
            
            # Update the bounds
            LB = max(LB,lowerbound) 
            UB = min(UB,upperbound)

            # add the optimality cut to the model
            for k in range(n):
                m.add_constraint(theta[k] >= lambda_l[k] - m.sum(pi_l[i,k]*x[i] for i in range(n)))

        # look for the next Pareto optimal solution
        runtime_iter= time.time() - start1    
        if lowerbound >= lb0 + epsilon: 
            minobj_list_pmcbd_mm.append(round(lowerbound,2))
            beta_list_pmcbd_mm.append(round(newbeta,2))
            time_pareto_pmcbd_mm.append(round(runtime_iter,2))
            num_pareto += 1 
            num_pareto_pmcbd_mm = num_pareto+1
            beta=newbeta
            lb0=lowerbound        
        if lowerbound < lb0 + epsilon:
            minobj_list_pmcbd_mm[-1]=round(lowerbound,2)
            beta_list_pmcbd_mm[-1]=round(newbeta,2)
            time_pareto_pmcbd_mm[-1]=round(runtime_iter,2)
            beta=newbeta    

        num_iter +=1
    
    return minobj_list_pmcbd_mm, beta_list_pmcbd_mm, num_pareto_pmcbd_mm, runtime_pmcbd_mm


In [17]:
# objective function for x
def f(x, q):
    x = x.copy()
    return x.dot(q).dot(x)

# gradient f
def df(x, q):
    x = x.copy()
    return x.dot(q)

In [19]:
n = 100
p_median = 10
epsilon = 1
time_limit = 3600

data = pd.read_csv(
    f"data/paper_data/GKD_data_{n}.txt",
    sep=r"\s+",
    header=None,
)
problem_data = ProblemData(data=data)
q = problem_data.distance_matrix

In [20]:
solver = BisectionMethod(ratio=0.3)
_,results = solver.optimise(problem=problem_data, p_median=p_median)

print(results)
min_distance = results["optimal_distance"].iloc[0]

                                    algorithm  p_median  optimal_distance  \
0  Adaptive Bisection Method for P-Dispersion        10          34.11047   

   runtime_seconds  iterations  ratio   status  
0         1.420655           7    0.3  optimal  


In [27]:
obj_min_pm_mm, obj_max_pm_mm, num_pareto_pm_mm, runtime_pm_mm = PM_Pareto(n,p_median,q,time_limit,min_distance,epsilon)
print(obj_min_pm_mm)
print(obj_max_pm_mm)
print(num_pareto_pm_mm)
print(runtime_pm_mm)

Infeasibility of epsilon constraint method is achieved.
[1273.74, 1304.65, 1325.75, 1330.7, 1376.33, 1381.65, 1386.92]
[5756.85, 6071.53, 6180.99, 6189.84, 6255.43, 6264.14, 6271.85]
7
56.652673959732056


In [28]:
LB_list_pmc_mm, beta_list_pmc_mm, num_pareto_pmc_mm, runtime_pmc_mm = PM_Cut(n,p_median,q,time_limit,min_distance,epsilon)
print(LB_list_pmc_mm)
print(beta_list_pmc_mm)
print(num_pareto_pmc_mm)
print(runtime_pmc_mm)


Infeasibility of cutting is achieved.
[1273.74, 1304.65, 1325.75, 1330.7, 1376.33, 1381.65, 1386.92]
[5756.85, 6071.53, 6180.99, 6189.84, 6255.43, 6264.14, 6271.85]
7
16.618731260299683


In [29]:

obj_min_pm_mm_balinski, obj_max_pm_mm_balinski, num_pareto_pm_mm_balinski, runtime_pm_mm_balinski = PM_Benders_balinski(n,p_median,q,time_limit,min_distance,epsilon)
print(obj_min_pm_mm_balinski)
print(obj_max_pm_mm_balinski)
print(num_pareto_pm_mm_balinski)
print(runtime_pm_mm_balinski)

Infeasibility of Benders with balinski cuts is achieved.
[1273.74, 1304.65, 1325.75, 1330.7, 1376.33, 1381.65, 1386.92]
[5756.85, 6071.53, 6180.99, 6189.84, 6255.43, 6264.14, 6271.85]
7
5.386752367019653


In [30]:
obj_min_pm_mm_bac, obj_max_pm_mm_bac, num_pareto_pm_mm_bac, runtime_pm_mm_bac = PM_branch_and_cut(n,p_median,q,time_limit,min_distance,epsilon)
print(obj_min_pm_mm_bac)
print(obj_max_pm_mm_bac)
print(num_pareto_pm_mm_bac)
print(runtime_pm_mm_bac)    

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Strategy_Search                     1
CPXPARAM_TimeLimit                               3600
Legacy callback                                  LD
Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
Clique table members: 1299.
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.01 sec. (1.53 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable B NodeID Parent  Depth

      0     0        0.0000    17                      0.0000       25      

CPLEX Error  1217: No solution exists.


Infeasibility of branch and cut is achieved.
[1273.74, 1304.65, 1325.75, 1330.7, 1376.33, 1381.65, 1386.92]
[5756.85, 6071.53, 6180.99, 6189.84, 6255.43, 6264.14, 6271.85]
7
54.187819719314575


In [25]:

obj_min_pm_mm, obj_max_pm_mm, num_pareto_pm_mm, runtime_pm_mm = PM_Benders(n,p_median,q,time_limit,min_distance,epsilon)
print(obj_min_pm_mm)
print(obj_max_pm_mm)
print(num_pareto_pm_mm)
print(runtime_pm_mm)

Infeasibility of Benders is achieved.
[1273.74, 1304.65, 1325.75, 1330.7, 1376.33, 1381.65, 1386.92]
[5756.85, 6071.53, 6180.99, 6189.84, 6255.43, 6264.14, 6271.85]
7
19.331406831741333


In [26]:
obj_min_pm_mm_optcuts, obj_max_pm_mm_optcuts, num_pareto_pm_mm_optcuts, runtime_pm_mm_optcuts = PM_Benders_optcuts(n,p_median,q,time_limit,min_distance,epsilon)
print(obj_min_pm_mm_optcuts)
print(obj_max_pm_mm_optcuts)
print(num_pareto_pm_mm_optcuts)
print(runtime_pm_mm_optcuts)

Infeasibility of Benders with optcuts is achieved.
[1273.74, 1304.65, 1325.75, 1330.7, 1376.33, 1381.65, 1386.92]
[5756.85, 6071.53, 6180.99, 6189.84, 6255.43, 6264.14, 6271.85]
7
36.02347445487976


## Run all instances

In [23]:
# objective function for x
def f(x, q):
    x = x.copy()
    return x.dot(q).dot(x)

# gradient f
def df(x, q):
    x = x.copy()
    return x.dot(q)

# ----- Parameters -----
n_values = [50]  # [50,100,150,200,250,300,350,400,450,500]
max_time = 3600
epsilon = 1

# Accumulate results as a Python list of dicts
all_results = []

for n in n_values:
    data = pd.read_csv(
        f"../data/paper_data/GKD_data_{n}.txt",
        sep=r"\s+",
        header=None,
    )
    problem_data = ProblemData(data=data)
    q = problem_data.distance_matrix

    p_values = [round(0.1 * n)]  # , round(0.2*n), round(0.3*n)]
    for p in p_values:
        solver = BisectionMethod(ratio=0.3)
        _, bisect_df = solver.optimise(problem=problem_data, p_median=p)

        # Extract mdis from the bisection output
        mdis = bisect_df["optimal_distance"].iloc[0]

        # -----------------------------
        # All commented methods
        # -----------------------------

        # PM_Pareto (bi-objective p-median & max-sum via Pareto enumeration/search)
        obj_min_pm_mm, obj_max_pm_mm, num_pareto_pm_mm, runtime_pm_mm = PM_Pareto(
            n, p, q, max_time, mdis, epsilon
        )

        # PM_Cut (cutting-plane approach)
        LB_list_pmc_mm, beta_list_pmc_mm, num_pareto_pmc_mm, runtime_pmc_mm = PM_Cut(
            n, p, q, max_time, mdis, epsilon
        )

        # PM_Benders_balinski (Benders decomposition with Balinski-style cuts)
        (
            obj_min_pm_mm_balinski,
            obj_max_pm_mm_balinski,
            num_pareto_pm_mm_balinski,
            runtime_pm_mm_balinski,
        ) = PM_Benders_balinski(n, p, q, max_time, mdis, epsilon)

        # PM_Benders (standard Benders)
        (
            minobj_list_pmcbd_mm,
            beta_list_pmcbd_mm,
            num_pareto_pmcbd_mm,
            runtime_pmcbd_mm,
        ) = PM_Benders(n, p, q, max_time, mdis, epsilon)

        # PM_branch_and_cut (BAC)
        (
            obj_min_pm_mm_bac,
            obj_max_pm_mm_bac,
            num_pareto_pm_mm_bac,
            runtime_pm_mm_bac,
        ) = PM_branch_and_cut(n, p, q, max_time, mdis, epsilon)

        # PM_Benders_optcuts (Benders with optimality cuts)
        (
            obj_min_pm_mm_optcuts,
            obj_max_pm_mm_optcuts,
            num_pareto_pm_mm_optcuts,
            runtime_pm_mm_optcuts,
        ) = PM_Benders_optcuts(n, p, q, max_time, mdis, epsilon)

        # Collect all outputs in one dict
        result = {
            "n": n,
            "p": p,

            # Pareto method
            "min-pm": obj_min_pm_mm,
            "max-pm": obj_max_pm_mm,
            "num-pm": num_pareto_pm_mm,
            "runtime-pm": runtime_pm_mm,

            # Cutting-plane method
            "min-pmc": LB_list_pmc_mm,
            "max-pmc": beta_list_pmc_mm,
            "num-pmc": num_pareto_pmc_mm,
            "runtime-pmc": runtime_pmc_mm,

            # Benders (Balinski)
            "min-balinski": obj_min_pm_mm_balinski,
            "max-balinski": obj_max_pm_mm_balinski,
            "num-balinski": num_pareto_pm_mm_balinski,
            "runtime-balinski": runtime_pm_mm_balinski,

            # Benders (standard)
            "min-pmcbd": minobj_list_pmcbd_mm,
            "max-pmcbd": beta_list_pmcbd_mm,
            "num-pmcbd": num_pareto_pmcbd_mm,
            "runtime-pmcbd": runtime_pmcbd_mm,

            # Branch-and-Cut
            "min-bac": obj_min_pm_mm_bac,
            "max-bac": obj_max_pm_mm_bac,
            "num-bac": num_pareto_pm_mm_bac,
            "runtime-bac": runtime_pm_mm_bac,

            # Benders with optimality cuts
            "min-optcuts": obj_min_pm_mm_optcuts,
            "max-optcuts": obj_max_pm_mm_optcuts,
            "num-optcuts": num_pareto_pm_mm_optcuts,
            "runtime-optcuts": runtime_pm_mm_optcuts,
        }

        all_results.append(result)

# Convert to DataFrame once at the end
bi_obj_df = pd.DataFrame(all_results)

# Save CSV to the current location
output_csv_path = os.path.join(os.getcwd(), "bi_objective_result.csv")
bi_obj_df.to_csv(output_csv_path, index=False)

print(f"Saved CSV: {output_csv_path}")


Infeasibility of epsilon constraint method is achieved.
Infeasibility of cutting is achieved.
Infeasibility of Benders with balinski cuts is achieved.
Infeasibility of Benders is achieved.
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Strategy_Search                     1
CPXPARAM_TimeLimit                               3600
Legacy callback                                  LD
Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
Clique table members: 659.
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.97 ticks)

        Nodes                                         Cuts/
   Node 

CPLEX Error  1217: No solution exists.


Infeasibility of branch and cut is achieved.
Infeasibility of Benders with optcuts is achieved.
Saved CSV: c:\Users\20103992\OneDrive - Curtin University of Technology Australia\Documents\academic\Curtin\models and algorithms\p-median and max-sum problem\Bi-objective-p-median-max-sum-problem\jupyter_code\bi_objective_result.csv
